In [ ]:
import cobra
import numpy as np
import scipy as sc
import cobra.test
model=cobra.test.create_test_model("textbook")

In [ ]:
import tools
model=tools.desdoblar(model,"desdoblado")

In [ ]:
tipos=tools.tiposReaccionesD(model)
reversibles=tipos[0]
irreversibles=tipos[1]

In [ ]:
S=cobra.util.create_stoichiometric_matrix(model,array_type="dense")
irreversibles_i=[]
model=tools.removeDeadEnds(model,S,reversibles,irreversibles,irreversibles_i)

In [ ]:
#Restricciones que le pongo

coeficientes=dict()
for i in irreversibles:
    r=model.reactions[i]
    coeficientes[r.forward_variable]=1
constraint=model.problem.Constraint(0,lb=1,ub=1)
model.add_cons_vars(constraint)
model.solver.update()
constraint.set_linear_coefficients(coefficients=coeficientes)

In [ ]:
#Coger la función

lista= np.ones(len(model.reactions))
valores=dict()
for i in range(len(lista)):
    r=model.reactions[i]
    valores[r]=lista[i]
model.objective=valores

In [ ]:
#Calcular la solucion del EFM
solucion=model.optimize(objective_sense="minimize")
valores=solucion.fluxes.values


In [ ]:
#Chequear si la EFM es una EFM
tools.check(valores,S)



In [ ]:
EFMS=[]
for i in range(1):
    coeficientes=dict()
    irreversibles_select= np.random.choice(irreversibles,10,replace=False)
    for i in irreversibles_select:
        r=model.reactions[i]
        coeficientes[r.forward_variable]=np.random.rand()
    constraint=model.problem.Constraint(0,lb=1,ub=1)
    model.add_cons_vars(constraint)
    model.solver.update()
    constraint.set_linear_coefficients(coefficients=coeficientes)
    lista= np.random.rand(len(model.reactions))
    valores=dict()
    for i in range(len(lista)):
        r=model.reactions[i]
        valores[r]=lista[i]
    model.objective=valores
    solucion1=model.optimize(objective_sense="minimize")
    valores=solucion1.fluxes.values
    soporte=[]
    noSoporte=[]
    for i in range(len(valores)):
        if valores[i]==0:
            noSoporte.append(i)
        else:
            soporte.append(i)
    reaccionesActivas=len(soporte)
    S1=np.delete(S,noSoporte,1)
    numeroEcuaciones=np.linalg.matrix_rank(S1)
    if (reaccionesActivas == numeroEcuaciones+1):
        print("TRUE")
        if soporte not in EFMS:
            EFMS.append(soporte)
    model.remove_cons_vars(constraint)

In [ ]:
import pandas
sol=pandas.DataFrame.from_dict([solucion.fluxes]).T
sol.to_csv('EFM_FBA.csv')